# Bayesian optimization assisted by CMA-ES

## Useage and Example

In [1]:
from bayes_opt import BayesianOptimization
import numpy as np

import warnings

import cma

In [11]:
class BayesianOptimization_CMAES(BayesianOptimization):
    def __init__(self, f, pbounds, random_state=None, verbose=2):
        super(BayesianOptimization_CMAES, self).__init__(f, pbounds, random_state=None, verbose=2)
        
    def acq_max_CMAES(self, ac, gp, y_max, bounds, random_state, n_warmup=100000, n_iter=250):
        x_seeds = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                   size=(bounds.shape[0]))
        options = {'ftarget': 1e-4,'bounds': bounds.T.tolist(),'maxiter':100000,'verb_log':0,'verb_time':False,'verbose':-9}
        res = cma.fmin(lambda x: 30-ac(x.reshape(1, -1), gp=gp, y_max=y_max), x_seeds, 0.25, options=options, 
               restarts =0, incpopsize=0, restart_from_best=False, bipop=False)
        x_max = res[0]
        return np.clip(x_max, bounds[:, 0], bounds[:, 1])
        
    def suggest(self, utility_function):
        """Most promissing point to probe next"""
        if len(self._space) == 0:
            return self._space.array_to_params(self._space.random_sample())
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self._gp.fit(self._space.params, self._space.target)
        suggestion = self.acq_max_CMAES(
            ac=utility_function.utility,
            gp=self._gp,
            y_max=self._space.target.max(),
            bounds=self._space.bounds,
            random_state=self._random_state
        )
        return self._space.array_to_params(suggestion)

In [12]:
def ff(**p):
     return (np.array(p['x']))**2+(np.array(p['y']))**2 + 1
    
optimizer = BayesianOptimization_CMAES(
    f=ff,
    pbounds={'x': (2.0, 4.0), 'y': (-3.0, 3.0)},
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)    

optimizer.maximize(
    init_points=1,
    n_iter=10,
    acq='ei',
    kappa=2.576,
    xi=0.0,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        |  11.36    |  3.218    |  0.07719  |
|  2        |  14.0     |  2.0      |  3.0      |
|  3        |  9.186    |  2.612    | -1.168    |
|  4        |  14.9     |  2.214    |  3.0      |
|  5        |  14.76    |  2.298    |  2.912    |
|  6        |  15.46    |  2.336    |  3.0      |
|  7        |  26.0     |  4.0      | -3.0      |
|  8        |  7.38     |  2.039    | -1.491    |
|  9        |  10.43    |  2.057    |  2.281    |
|  10       |  17.03    |  3.045    |  2.599    |
|  11       |  14.12    |  3.402    |  1.243    |
